In [3]:
from transformers import pipeline, set_seed, GPT2Tokenizer
from torch import tensor

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

generator = pipeline('text-generation', model = 'gpt2', tokenizer = tokenizer)
set_seed = 0

In [7]:
generator.model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
## Few-shot learning

In [8]:
print(generator("""Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment:""", top_k = 2, temperature = 0.1, max_length = 55)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment: Negative



In [9]:
print(generator("""Question/Answering
C: Google was founded in the 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control and 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julient Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American footbal team based in the New York metropolitan area. The Jets compete in the National Footbal League (NFL) as a member club of the league's American Football Conference (AFC) East division
Q: What division do the Jets play in?
A:""", top_k = 2, max_length = 215, temperature = 0.5)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: Google was founded in the 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control and 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julient Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American footbal team based in the New York metropolitan area. The Jets compete in the National Footbal League (NFL) as a member club of the league's American Football Conference (AFC) East division
Q: What division do the Jets play in?
A: The AFC East. The


In [ ]:
## Zero Shot Learning

In [11]:
# Same question as before, with no previous examples in Zero-shot learning. Still works
print(generator("""Question/Answering
C: The New York Jets are a professional American footbal team based in the New York metropolitan area. The Jets compete in the National Footbal League (NFL) as a member club of the league's American Football Conference (AFC) East division
Q: What division do the Jets play in?
A:""", top_k = 3, max_length = 80, temperature = 0.5)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: The New York Jets are a professional American footbal team based in the New York metropolitan area. The Jets compete in the National Footbal League (NFL) as a member club of the league's American Football Conference (AFC) East division
Q: What division do the Jets play in?
A: The Jets play in the AFC East, the AFC


In [12]:
# Zero-shot doesn't work as much with the sentiment analysis example
print(generator("""Sentiment Analysis
Text: This new music video was so good
Sentiment:""", top_k = 2, temperature = 0.1, max_length = 55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: This new music video was so good
Sentiment: This new music video was so good
Sentiment: This new music video was so good
Sentiment: This new music video was so good
Sentiment: This new music video was


In [ ]:
# Zero-shot abstractive summarization

In [13]:
to_summarize = """This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to neccesary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architechure. We will then move into GPT-2 models as well as fine-tunning these models on custom corpora.
GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning model like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.
The Natural Language Processing with Next-Generation Transformer Architecture series of online trainings provides a comprehensive overview of the state-of-the-art natural language processing (NLP) models including GPT and BERT which are derived from the modern attention-driven transformer architecture and the applications these models are used to solve today. All of the trainings in the series blend theory and application through the combination of visual mathematical explanations, strainghtforward applicable Python examples within hands-on jupyter notebook demos, and comprehensive case studies featuring modern problems solvable by NLP models. (Note that at any given time, only a subset of these classes will be scheduled and open for registration.)
"""

In [18]:
print(generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""",
    max_length = 512, top_k = 5, temperature = 0.5, no_repeat_ngram_size = 2
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summarization Task:
This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to neccesary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architechure. We will then move into GPT-2 models as well as fine-tunning these models on custom corpora.
GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning model like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.
The Natural Language Processing with Next-Generation Transformer Architecture series of online trainings provides a comprehensive overview of the state-of-the-art natural language processing (NLP) models including GPT and BER